### Find the vertices of the `stoich_subspace()` 

The stoichiometric subspace is a region in concentration space wherein all feasible concentrations must lie.  It forms the upper bounds in concentration space in which the true attainable region must lie. This region originates from the feed point and it must obey reaction stoichiometry. Thus, it is typically much larger than the set of 'attainable' points. For n components participating in d reactions, the true AR must reside in a d-dimensional subspace in $R^n$. And, it follows that the dimension d, is the number of linearly independent reactions (rank(A)) present in a system. 

For any specified feed vector $(C_f)$ and extent of reaction $(\varepsilon)$, the set of stoichiometrically compatible concentrations must lie within a region spanned by the feed vector and stoichiometric coefficient $(A)$ vector in concentration space. As a consequence of the physical meaning of achievable concentrations, mass balance and non-negativity constraints are imposed, further restricting the set of physically attainable concentrations. This implies that every point $c_i$ within the stoichiometric subspace must reside in the positive orthant of the 'real' concentration space. Furthermore, it must be stoichiometrically compatible with the given reaction stoichiometry. 

The stoichiometric subspace is denoted by the letter S and it obeys the following vector equation parameterized by the extent of reaction;

$C = C_f + A\varepsilon$

Where;

$C_f$ : feed vector
    
$A$ : stoichiometric coefficient A, with the size ${n x d}$
    
$\varepsilon$ : extent vector corresponding to each reaction taking part in the system

Computing a stoichiometric subspace for a single feed is done by identifying the bounding constraints in extent space that form the stoichiometric subspace. Vertex enumeration is employed to find the extreme vertices in extent space that constitute the feasible region. Consequently, finding the vertices of the convex polytope, S, in $R^n$ i.e. computing the convex hull of the stoichiometric subspace polytope described by the inequality constraints;

$C_f + A\varepsilon \geq 0 $

The ` con2vert()` function in the `artools package` is deployed in solving the vertex enumeration problem. Since the stoichiometric subspace is a function of both the reaction stoichiometry and the feed point, thus, the shape of the stoichiometric subspace will change if the feed point is changed. This implies that each feed point has an associated stoichiometrci subspace, S. Hence, it can be deduced that; the set of concentrations belonging to the overall stoichiometric subspace, $S_{tot}$, may be found by computing convex hull of the individual stoichiometric subspaces;

$S_{tot} = conv ({S_1, S_2, S_3, ..., S_N})$

Since the overall stoichiometric subspace is found by mixing between individual stoichiometric subspaces, the dimension of the subspace is increased. Thus, the dimension of the AR exceeds the no. of independent reactions. 

### Imports

In [1]:
import scipy as sp
import scipy.integrate
import artools
import matplotlib.pyplot as plt
from scipy import linalg
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
from __future__ import division

%matplotlib inline

In [3]:
def stoich_subspace(Cf0s, stoich_mat):
    """Compute the stoichiometric subspace from a stoichometric coefficient
    matrix and multiple feed points.

    Parameters:
        stoich_mat  (n x d) array. Each row in stoich_mat corresponds
                    to a component and each column corresponds to a
                    reaction.
        Cf0s        (M x n) matrix of M feeds. Each row in Cf0s is a feed
                    corresponding to a component row in stoich_mat.

    Returns:
        Cs     (L x n) array. The vertices of the stoichiometric subspace in
               concentration space.
        Es     (L x d) array. The vertices of the stoichiometric subspace in
               extent space.
    """
    
    
    '''if Cf0s is not a list:
        Cf0s = [Cf0s]

        if CF0s.ndim == 2 and Cf0.shape[0]>1 and CF0s.shape[1]>1:
                unpack matrix out of list
    '''
    
    # create empty lists
    all_Es = []
    all_Cs = []
    
    # if user input is not a list, then convert into a list 
    if not isinstance(Cf0s, list) and not Cf0s.shape[0] > 1 and not Cf0s.shape[1] > 1:
        # put it in a list 
        Cf0s = [Cf0s]
        
    for Cf0 in Cf0s:
        # check if Cf0 is a column vector with ndim=2, or a (L,) array with
        # ndim=1 only
        if Cf0.ndim == 2:
            Cf0 = Cf0.flatten() # converts into (L,)
            
            # raise an error if the no. of components is inconsistent
            if len(Cf0) != stoich_mat.shape[0]:
                raise Exception("No. of components is not consistent with the no. of rows in stoichiometric matrix")
            
        # always treat stoich_mat as a matrix for consistency, convert if not
        if stoich_mat.ndim == 1: 
            # converts a 'single rxn' row into column vector  
            stoich_mat = stoich_mat.reshape((len(stoich_mat), 1)) 

        # check if it's a single rxn 
        if stoich_mat.shape[1] == 1:

            stoich_mat = stoich_mat.flatten()

            # calculate the limiting requirements
            limiting = Cf0/ stoich_mat

            # only choose negative coefficients as these indicate reactants
            k = limiting < 0.0

            # calc maximum extent based on limiting reactant and calc C
            e_max = sp.fabs(max(limiting[k])) # shouldn't we take the min?? we take max because of the negative

            C = Cf0 + stoich_mat*e_max

            # form Cs and Es and return
            Cs = sp.vstack([Cf0, C])
            Es = sp.array([[0., e_max]]).T

            return Cs, Es

        # extent associated with each feed vector
        Es = artools.con2vert(- stoich_mat, Cf0) 

        Cs = (Cf0[:, None] + sp.dot(stoich_mat, Es.T)).T # what does this slice, 'Cf0[:, None]' do?

        # vertices for each feed and stoich_mat in extent and concentration space
        all_Es.append(Es) 
        all_Cs.append(Cs)

        # store all vertices in one array
        all_Es_mat = sp.vstack(all_Es)
        all_Cs_mat = sp.vstack(all_Cs)

    return all_Es_mat, all_Cs_mat 